<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/shaptonstall/breaker_ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/capstone

/content/drive/MyDrive/Github/capstone


In [3]:
%ls

 Capstone.ipynb           pd2       pd.png                               trivium_scaler.pkl
 data_exploration.ipynb   pd2.png   README.md                            trivium_scorecard.csv
 glicko2.py               pd3      'trivium_full_column_names (1).csv'   Webscraping.xlsx
 pd                       pd3.png   trivium_pca.pkl


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/Github/capstone')

In [5]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
from sklearn.decomposition import PCA, KernelPCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import FactorAnalysis
import joblib
from glicko2 import Glicko2, Rating

In [6]:
#load models if needed
scaler = joblib.load('trivium_scaler.pkl')
pca= joblib.load('trivium_pca.pkl')

In [7]:
#load files
trivium= pd.read_csv('trivium_scorecard.csv')
webscraping = pd.read_excel('Webscraping.xlsx', sheet_name=None)
webscraping.keys()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


dict_keys(['Events and8', 'Events', 'Events WDSF', 'Battles Trivium', 'Battles and8 Threefold', 'Battles and8 Single Slider', 'Battles and8 Round by Round', 'Battles and8 Traditional', 'Research Questions', 'Notes', 'Notes Columns'])

In [8]:
#Load battle winners
#threefold
threefold = webscraping['Battles and8 Threefold']
threefold.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category', 'breaker 1': 'breaker 1 (red)', 'breaker 2': 'breaker 2 (blue)'}, inplace=True)
threefold[['Event', 'Category']] = threefold[['Event', 'Category']].fillna(method='ffill')
#Trivium
trivium=webscraping['Battles Trivium']
trivium.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
trivium[['Event', 'Category']]= trivium[['Event', 'Category']].fillna(method='ffill')
#traditional
traditional = webscraping['Battles and8 Traditional']
traditional.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
traditional[['Event', 'Category']] = traditional[['Event', 'Category']].fillna(method='ffill')

In [9]:
tr=traditional[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()
th=threefold[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()
tv=trivium[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()

dataframes = [('traditional', tr), ('threefold', th), ('trivium', tv)]

# Create an empty list to store modified dataframes
appended_dfs = []

# Iterate to create a combined dataframe
for name, dframe in dataframes:
    dframe['Original_DF'] = name
    dframe['Battle_ID'] = dframe.index
    appended_dfs.append(dframe)
appendedBattles = pd.concat(appended_dfs, ignore_index=True)


In [28]:
appendedBattles.head(5)

,Event,Category,breaker 1 (red),breaker 2 (blue),winner,Original_DF,Battle_ID,Date
1914,WDSF World Championship 2021 - Paris,B-Girls Top 32 Round Robin Group F,Miimo,671,671,trivium,811,2021-12-04 00:00:00
1916,WDSF World Championship 2021 - Paris,B-Girls Top 32 Round Robin Group F,Mery Berry,671,671,trivium,813,2021-12-04 00:00:00
1587,The IWGA World Games 2022,B-Girls Top 8 Knockout,671,Tiff,671,trivium,484,2022-07-10 00:00:00
1598,The IWGA World Games 2022,B-Girls Top 16 Round Robin Group C,671,Alessandrina,671,trivium,495,2022-07-10 00:00:00
1599,The IWGA World Games 2022,B-Girls Top 16 Round Robin Group C,671,Pauline,671,trivium,496,2022-07-10 00:00:00


In [10]:
eventsAnd8=webscraping['Events and8'][1:].copy()
new_header= eventsAnd8.iloc[0]
eventsAnd8=eventsAnd8.drop(1)
eventsAnd8.columns = new_header
eventsAnd8.rename(columns={'Name':'Event'}, inplace=True)
eventsAnd8.drop_duplicates(subset=['Event'], inplace=True)

eventsWDSF = webscraping['Events WDSF'][1:].copy()
new_header = eventsWDSF.iloc[0]
eventsWDSF = eventsWDSF.drop(1)
eventsWDSF.columns = new_header
eventsWDSF.rename(columns={'Name': 'Event'}, inplace=True)
eventsWDSF.drop_duplicates(subset=['Event'], inplace=True)

Events = webscraping['Events'][1:].copy()
new_header = Events.iloc[0]
Events = Events.drop(1)
Events.columns = new_header
Events.rename(columns={'Name':'Event'}, inplace=True)
Events.drop_duplicates(subset=['Event'], inplace=True)

combined_events = pd.concat([eventsAnd8, eventsWDSF, Events], ignore_index=True)

In [11]:
#add battle date
appendedBattles=pd.merge(appendedBattles, combined_events[['Date', 'Event']], on='Event', how='left')


In [12]:
# Elo Rating function
def calculate_elo(winner_elo, loser_elo, k=32):
    expected_winner = 1 / (1 + 10 ** ((loser_elo - winner_elo) / 400))
    expected_loser = 1 / (1 + 10 ** ((winner_elo - loser_elo) / 400))
    new_winner_elo = winner_elo + k * (1 - expected_winner)
    new_loser_elo = loser_elo - k * expected_loser
    return new_winner_elo, new_loser_elo

In [13]:
# Initialize participant Elo ratings
appendedBattles.sort_values(by=['winner', 'Date'], inplace=True)
participants = list(set(appendedBattles['breaker 1 (red)']).union(set(appendedBattles['breaker 2 (blue)'])))
elo_ratings = dict.fromkeys(participants, 1500)


In [ ]:
# @title
# # Function to update the Elo ratings based on the DataFrame
# def update_elo_from_df(df, dictionary):
#     for index, row in df.iterrows():
#         red_breaker = row['breaker 1 (red)']
#         blue_breaker = row['breaker 2 (blue)']
#         winner = row['winner']

#         # Find out who is the loser
#         loser = red_breaker if winner == blue_breaker else blue_breaker

#         # Get the current Elo ratings
#         red_elo = dictionary[red_breaker]
#         blue_elo = dictionary[blue_breaker]
#         print("original ranking",red_elo, blue_elo)

#         # Update Elo ratings based on the match result
#         if winner == red_breaker:
#             new_red_elo, new_blue_elo = calculate_elo(red_elo, blue_elo)
#         else:
#             new_blue_elo, new_red_elo = calculate_elo(blue_elo, red_elo)

#         dictionary[red_breaker] = new_red_elo
#         dictionary[blue_breaker] = new_blue_elo
#         print("new ranking",dictionary[red_breaker], dictionary[blue_breaker] )
#     return dictionary





In [14]:
eloRankDF=pd.DataFrame()
for index, row in appendedBattles.iterrows():
  temp_dict=row.to_dict()
  red_breaker = row['breaker 1 (red)']
  blue_breaker = row['breaker 2 (blue)']
  winner = row['winner']

  # Find out who is the loser
  loser = red_breaker if winner == blue_breaker else blue_breaker

  # Get the current Elo ratings
  red_elo = elo_ratings[red_breaker]
  blue_elo = elo_ratings[blue_breaker]
  temp_dict['orig_red_elo'] = red_elo
  temp_dict['orig_blue_elo'] = blue_elo
  # Update Elo ratings based on the match result
  if red_elo == blue_elo:
    temp_dict['Favorite']= "draws"
  elif red_elo > blue_elo:
    temp_dict['Favorite']= red_breaker
  else:
    temp_dict['Favorite']= blue_breaker

  if winner == red_breaker:
      new_red_elo, new_blue_elo = calculate_elo(red_elo, blue_elo)
  else:
      new_blue_elo, new_red_elo = calculate_elo(blue_elo, red_elo)
  elo_ratings[red_breaker] = new_red_elo
  elo_ratings[blue_breaker] = new_blue_elo
  temp_dict['new_red_elo'] = new_red_elo
  temp_dict['new_blue_elo'] = new_blue_elo
  new_row = pd.DataFrame([temp_dict])
  eloRankDF = pd.concat([eloRankDF, new_row], ignore_index=True)


In [15]:
 eloRankDF.head(5)

,Event,Category,breaker 1 (red),breaker 2 (blue),winner,Original_DF,Battle_ID,Date,orig_red_elo,orig_blue_elo,Favorite,new_red_elo,new_blue_elo
0,WDSF World Championship 2021 - Paris,B-Girls Top 32 Round Robin Group F,Miimo,671,671,trivium,811,2021-12-04,1500.000000,1500.0,draws,1484.000000,1516.000000
1,WDSF World Championship 2021 - Paris,B-Girls Top 32 Round Robin Group F,Mery Berry,671,671,trivium,813,2021-12-04,1500.000000,1516.0,671,1484.736307,1531.263693
2,The IWGA World Games 2022,B-Girls Top 8 Knockout,671,Tiff,671,trivium,484,2022-07-10,1531.263693,1500.0,671,1545.827820,1485.435873
3,The IWGA World Games 2022,B-Girls Top 16 Round Robin Group C,671,Alessandrina,671,trivium,495,2022-07-10,1545.827820,1500.0,671,1559.729526,1486.098294
4,The IWGA World Games 2022,B-Girls Top 16 Round Robin Group C,671,Pauline,671,trivium,496,2022-07-10,1559.729526,1500.0,671,1573.005662,1486.723865


In [16]:
sorted_elo=dict(sorted(elo_ratings.items(), key=lambda item: item[1], reverse=True))
sorted_elo=pd.DataFrame.from_dict(sorted_elo, orient='index', columns=['Rank']).reset_index()
sorted_elo.columns= ['Breaker', 'Elo Rank']
sorted_elo.head(5)

,Breaker,Elo Rank
0,Shigekix,1842.074324
1,Victor,1839.855111
2,Sunny,1784.672083
3,Ami,1759.608864
4,Phil Wizard,1734.044628


In [18]:
glicko_env = Glicko2()

In [26]:
matches = appendedBattles

# Create initial player ratings
players = {name: glicko_env.create_rating() for name in pd.unique(matches[['breaker 1 (red)', 'breaker 2 (blue)']].values.ravel('K'))}

for _, row in matches.iterrows():
    # Get the player names and ratings
    player1_name = row['breaker 1 (red)']
    player2_name = row['breaker 2 (blue)']
    winner_name = row['winner']

    player1_rating = players[player1_name]
    player2_rating = players[player2_name]

    # Determine match outcome
    if player1_name == winner_name:
        outcome = (1, 0)
    else:
        outcome = (0, 1)

    # Create period for each player
    period1 = glicko_env.period()
    period2 = glicko_env.period()

    period1.add_player(player1_rating, [outcome[0]], [player2_rating])
    period2.add_player(player2_rating, [outcome[1]], [player1_rating])

    # Calculate new ratings
    new_ratings1 = period1.rate()
    new_ratings2 = period2.rate()

    # Update the ratings dictionary
    players[player1_name] = new_ratings1[player1_rating]
    players[player2_name] = new_ratings2[player2_rating]

# Output the final ratings
for player_name, rating in players.items():
    print(f'{player_name}: Rating {rating.mu}, RD {rating.phi}, Volatility {rating.sigma}')

AttributeError: 'Glicko2' object has no attribute 'period'